# Preprocessing

In [ ]:
import mne

def reject_criteria(x):

  max_condition = np.max(x, axis = 1) > 1e-4 # Condition for if the maximum value of the array is above 0.0001 V
  min_condition = np.min(x, axis = 1) < -1e-4 # Condition for if the minimum value of the array is below -0.0001 V

  # Returns the result for the given array. Whether to reject or not the array if a condition is true.
  return ((max_condition.any() or min_condition.any()), ["max amp", "min amp"])

# This function takes in the file path of an .edf file and preprocesses the file accordingly to our needs
def read_data(file_path):

  # To turn the edf file into a raw object so that mne functions can be applied to the data
  data = mne.io.read_raw_edf(file_path, preload = True)

  # This setting sets the raw object to understand that the data is on eeg
  data.set_eeg_reference()

  # Bandpass filters are applied using this function on the data. Low pass filter of 0.1 Hz and High pass filter of 70 Hz is used
  data.filter(l_freq = 0.1, h_freq = 70)

  # Notch filter of 50 Hz is applied on the data using the function below
  data.notch_filter(50)

  # The channels used for each patient can vary from 20 to 22 channels.
  # The conditions below are used to removed targeted channels as doing so will improve the performance of the deep learning model
  # Channels 'EEG 23A-23R' and 'EEG 24A-24R' are reference channels, while the other channels are selected as removing them improved performance
  if data.info['nchan'] == 22:
    data.drop_channels(['EEG 23A-23R', 'EEG 24A-24R', 'EEG A2-A1', 'EEG C3-LE', 'EEG F4-LE'])
  if data.info['nchan'] == 20:
    data.drop_channels(['EEG A2-A1', 'EEG C3-LE', 'EEG F4-LE'])

  data.rename_channels({
    'EEG Fp1-LE': 'Fp1', 'EEG Fp2-LE': 'Fp2',
    'EEG F7-LE': 'F7', 'EEG F3-LE': 'F3', 'EEG Fz-LE': 'Fz', 'EEG F8-LE': 'F8',
    'EEG T3-LE': 'T3', 'EEG Cz-LE': 'Cz', 'EEG C4-LE': 'C4', 'EEG T4-LE': 'T4',
    'EEG T5-LE': 'T5', 'EEG P3-LE': 'P3', 'EEG Pz-LE': 'Pz', 'EEG P4-LE': 'P4', 'EEG T6-LE': 'T6',
    'EEG O1-LE': 'O1', 'EEG O2-LE': 'O2'
    })
  data.set_montage(mne.channels.make_standard_montage("standard_1020"))

  # Independent Component Analysis (ICA) is applied here to help reduce the noise in the signals
  ica = mne.preprocessing.ICA(random_state = 42, n_components = 13)

  # The ICA is fitted to a copy of the data that has is filtered using a low pass filter of 1 Hz as ICA does not work well with low frequencies
  ica.fit(data.copy().filter(l_freq = 1.0, h_freq = None))

  data = ica.apply(data.copy()) # The fitted ICA is then applied on the data

  # Segmentation of signals is done by creating fixed length epochs that are 10 seconds each.
  epochs = mne.make_fixed_length_epochs(data, duration = 5, overlap = 2)

  # Using the reject criteria function made above, we can detect segments of the signals that are noisy and drop them from the data we want to use, cleaning the dataset
  epochs.drop_bad(reject = dict(eeg=reject_criteria))

  # This function returns the data from epoches in the form of an np array
  array = epochs.get_data()

  # Returns the preprocessed data
  return array

# Using list comprehension, each of the file paths are given in the read data function and their respective preprocessed data is extracted and stored in the proper list
h_epochs_list = [read_data(i) for i in h_file_paths]
mdd_epochs_list = [read_data(i) for i in mdd_file_paths]

h_epochs_labels = [len(i) * [0] for i in h_epochs_list]
mdd_epochs_labels = [len(i) * [1] for i in mdd_epochs_list]

data_list = h_epochs_list + mdd_epochs_list
label_list = h_epochs_labels + mdd_epochs_labels

data_array = np.vstack(data_list)

label_array = np.hstack(label_list)

from sklearn.preprocessing import StandardScaler

# We need to reshape the 3D array into a 1D array too allow the scaler to fit to all of the data points
original_shape = data_array.shape # Saving the original shape of the array

data_array = data_array.reshape(-1, 1) # Reshaping the array into a 1D array

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_array) # Fitting the scaler and saving the scaled data array in a new variable

data_array = scaled_data.reshape(original_shape)

# Feature Extraction

In [ ]:
mean_signals = [] # list to hold the means of the data

for d in data_array:
  mean_signals.append(np.mean(d, axis = 0))

from scipy import signal

X_data = [] # This will be the list that will hold the image data
sequence_length = 0
image_shape = []

import cv2

# The loop goes through each of the segment and adds the spectrogram data of the segments to the list
for d in mean_signals:

  # To create the spectrogram, the STFT data is first generated from our data
  frequencies, times, stft_data = signal.spectrogram(x = d, fs = 256, window=('tukey', 0.25), nperseg=32, noverlap=16, nfft=32)

  # The generated stft data is then resized accordingly such that it holds proper image data as a spectrogram
  resized_spectrogram = np.abs(stft_data)
  resized_spectrogram = resized_spectrogram / np.max(resized_spectrogram)

  resized_spectrogram = cv2.resize(resized_spectrogram, dsize=(75, 17), interpolation=cv2.INTER_CUBIC)

  split_array = np.split(resized_spectrogram, 3, axis=1)
  spectrogram_array = np.array(split_array)

  X_data.append(spectrogram_array)

X_data = np.array(X_data)

# Model

In [ ]:
# As the data of each patient is stored in their respective file, we must first be able to access them individually
# To do so we need to get their relative file paths and store them in a list for easy access

# The os library is imported as it's functions are needed to extract and handle file paths
import os

folder_path = "/mumtaz_dataset" # This is the directory path where files of the dataset is located

# Using list comprehensions, a list is made where each element is a file path to a file in the dataset

# The code below creates a list with the name of all the files from the dataset IF the file is checked to be a file
file_paths = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# The code below then joins the name of the files from the dataset with the path of the directory.
# This creates a list of the all relative file path for the dataset files
file_paths = [os.path.join(folder_path, f) for f in file_paths]

In [ ]:
# The Healthy and Major Depressive Disorder files should be seperated since they will be labeled differently for our deep learning model to recognize
# These files are seperated to allow labeling to be much simpler

# Seperate lists for the healthy (h) and major depressive disorder (mdd) data
h_file_paths = []
mdd_file_paths = []

# This loop goes through the list containing all of the file paths of the dataset files and by checking the file names,
# places the relative file paths accordingly to the respective list.
# Eyes Closed (EC) files are used as the Stimuli chosen to teach the model with
for i in file_paths:

  # Gets the file name from the relative file path as it splits the path where there is a '/' and
  file_name = i.split('/')[-1]

  # If the file name has 'EC' in it, it means that the patient had their eyes closed and it continues
  if ('EC' in file_name):

    # If there is a 'H' in the file name, it means that it is a file for a Healthy patient and adds that file's relative path to the healthy list
    if ('H' in file_name):
      h_file_paths.append(i)

    # If there is a 'M' in the file name, it means that it is a file for a Major Depressive Disorder patient and adds that file's relative path to the major depressive disorder list
    if ('M' in file_name):
      mdd_file_paths.append(i)

In [ ]:
# All of the files are in the .edf file format. .edf files are able to hold a lot of useful information for our needs,
# thus to effectively make use of that data, the mne library is imported
import mne

# This function is used in the next function. It is used to help clean what is considered noisy signal from the data
# Amplitudes for EEG signals ranges from 0 - 100 microVolts (µV) (https://www.researchgate.net/publication/339736502_Classification_of_Brainwaves_for_Sleep_Stages_by_High-Dimensional_FFT_Features_from_EEG_Signals)
# Since the data type of the signals is in V, we will consider any values above 0.0001 V (1e-4) or below -0.0001 V (-1e-4) to be noise
def reject_criteria(x):

  max_condition = np.max(x, axis = 1) > 1e-4 # Condition for if the maximum value of the array is above 0.0001 V
  min_condition = np.min(x, axis = 1) < -1e-4 # Condition for if the minimum value of the array is below -0.0001 V

  # Returns the result for the given array. Whether to reject or not the array if a condition is true.
  return ((max_condition.any() or min_condition.any()), ["max amp", "min amp"])

# This function takes in the file path of an .edf file and preprocesses the file accordingly to our needs
def read_data(file_path):

  # To turn the edf file into a raw object so that mne functions can be applied to the data
  data = mne.io.read_raw_edf(file_path, preload = True)

  # This setting sets the raw object to understand that the data is on eeg
  data.set_eeg_reference()

  # Bandpass filters are applied using this function on the data. Low pass filter of 0.1 Hz and High pass filter of 70 Hz is used
  data.filter(l_freq = 0.1, h_freq = 70)

  # Notch filter of 50 Hz is applied on the data using the function below
  data.notch_filter(50)

  # The channels used for each patient can vary from 20 to 22 channels.
  # The conditions below are used to removed targeted channels as doing so will improve the performance of the deep learning model
  # Channels 'EEG 23A-23R' and 'EEG 24A-24R' are reference channels, while the other channels are selected as removing them improved performance
  if data.info['nchan'] == 22:
    data.drop_channels(['EEG 23A-23R', 'EEG 24A-24R', 'EEG Fz-LE', 'EEG Cz-LE', 'EEG Pz-LE'])
  if data.info['nchan'] == 20:
    data.drop_channels(['EEG Fz-LE', 'EEG Cz-LE', 'EEG Pz-LE'])

  # Independent Component Analysis (ICA) is applied here to help reduce the noise in the signals
  ica = mne.preprocessing.ICA(random_state = 42, n_components = 13)

  # The ICA is fitted to a copy of the data that has is filtered using a low pass filter of 1 Hz as ICA does not work well with low frequencies
  ica.fit(data.copy().filter(l_freq = 1.0, h_freq = None))

  data = ica.apply(data.copy()) # The fitted ICA is then applied on the data

  # Segmentation of signals is done by creating fixed length epochs that are 10 seconds each.
  epochs = mne.make_fixed_length_epochs(data, duration = 10, overlap = 2)

  # Using the reject criteria function made above, we can detect segments of the signals that are noisy and drop them from the data we want to use, cleaning the dataset
  epochs.drop_bad(reject = dict(eeg=reject_criteria))

  # This function returns the data from epoches in the form of an np array
  array = epochs.get_data()

  # Returns the preprocessed data
  return array

In [ ]:
# Using list comprehension, each of the file paths are given in the read data function and their respective preprocessed data is extracted and stored in the proper list
h_epochs_list = [read_data(i) for i in h_file_paths]
mdd_epochs_list = [read_data(i) for i in mdd_file_paths]

In [ ]:
# Using the preprocessed data lists, their respective labels are made. 0 meaning Healthy patients and 1 meaning Major Depressive Disorder patients
# These values are stored in their appropriate lists
h_epochs_labels = [len(i) * [0] for i in h_epochs_list]
mdd_epochs_labels = [len(i) * [1] for i in mdd_epochs_list]

In [ ]:
# Now that we have the data and labels for both Healthy and Major Depressive Disorder data, we can merge the data together to a collective list
# Now we have a list containing all of our data and another list containing all of our labels.
# The same indexes of the lists are related, for example index 1 of the data list has its corespondiing label of index 1 in the label list
# The reason this combining step is done is so that the data can be randomly split properly
data_list = h_epochs_list + mdd_epochs_list
label_list = h_epochs_labels + mdd_epochs_labels

In [ ]:
# At the moment the data and labels are stored within lists. We need to convert them into np arrays such that it can be manipulated more efficiently using numpy functions
# The numpy library is imported to make use of its function
import numpy as np

# Stacking the data list vertically, allowing a 3 dimensional array to be made of the data.
# First dimension is the number of segments of the data
# Second dimension is the number of channels kept for each segment
# Third dimension is time. As the sampling rate is 256 Hz, over 10 seconds there are 2560 data points for each channel
data_array = np.vstack(data_list)

# Stacking the data horizontally, creating a 1 dimensional array containing all of the labels for each of the segments
label_array = np.hstack(label_list)

In [ ]:
# Another preprocessing step to take is to scale the data using z-score normalization
# Doing so allows all of the preprocessed data to have the same scale and prevent certain data points from dominating over others
# The StandardScaler function from sklearn is used to apply this step
from sklearn.preprocessing import StandardScaler

# We need to reshape the 3D array into a 1D array too allow the scaler to fit to all of the data points
original_shape = data_array.shape # Saving the original shape of the array

data_array = data_array.reshape(-1, 1) # Reshaping the array into a 1D array

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_array) # Fitting the scaler and saving the scaled data array in a new variable

#data_array = scaled_data.reshape(1853, 17, 2560)
# Reshaping the scaled data array back to the original shape
data_array = scaled_data.reshape(original_shape)

In [ ]:
# Having 17 channels to extract features from can lead to difficulties in creating the proper input to our deep learning model
# Therefore, the mean of the 17 channels will be used instead

mean_signals = [] # list to hold the means of the data

# The loop below loops over the data array and gets the mean value between all 17 channels for that point of time
# This results in a 2 dimensional array,
# with the first dimension being the segments of the data and the second dimension being the time dimension, with the values of each point being the mean of the 17 channels
for d in data_array:
  mean_signals.append(np.mean(d, axis = 0))

In [ ]:
# The feature that will be extracted from the preprocessed data is an image, specifically a spectrogram
# This can be achieved using the functions in the scipy library
from scipy import signal

n_freq_bins = 256
n_time_bins = 384

X_data = [] # This will be the list that will hold the image data

# The loop goes through each of the segment and adds the spectrogram data of the segments to the list
for d in mean_signals:

  # To create the spectrogram, the STFT data is first generated from our data
  frequencies, times, stft_data = signal.spectrogram(x = d, fs = 256, window = signal.windows.tukey(64, 0.25))

  # The generated stft data is then resized accordingly such that it holds proper image data as a spectrogram
  resized_spectrogram = np.abs(stft_data)
  resized_spectrogram = resized_spectrogram[:n_freq_bins, :n_time_bins]
  resized_spectrogram = resized_spectrogram / np.max(resized_spectrogram)

  X_data.append(resized_spectrogram) # Adds to the data list

X_data = np.array(X_data) # Converts the list into a np array

In [ ]:
# As the features are now prepared to develop the model, the data needs to be split into a training and test datasets
# This will be done using the train test split function
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, label_array, stratify = label_array, test_size = 0.2, random_state = 42)

In [ ]:
# For the deep learning model tensorflow will be used as the library to create it
import tensorflow as tf
from tensorflow import keras

# This function below creates the deep learning model based its architecture
def create_model():

  # As images are being used as the input to the model, 2D convolutional layers are utilized
  # This architecture is obtained through trial and error, whilst using grid search to tune it's hyperparameters
  model = keras.Sequential([
    keras.layers.Conv2D(128, kernel_size = (3, 3), activation = 'relu', input_shape = (33, 45, 1)),
    keras.layers.MaxPooling2D(pool_size = (2, 2)),
    keras.layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(pool_size = (2, 2)),
    keras.layers.Conv2D(32, kernel_size = (2, 2), activation = 'relu'),
    keras.layers.MaxPooling2D(pool_size = (2, 2)),
    keras.layers.Reshape((2, 4 * 32)), # Reshaping the data from Pooling to be suitable for LSTM
    keras.layers.LSTM(128), # LSTM is used to better understand temporal patterns found in the data
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(2, activation = 'sigmoid') # A final dense layer with an output of 2 is used as our model has a binary output (Healthy and Major Depressive Disorder)
  ])

  # Adam is used as the optimizer and sparse categorical crossentropy is used as the loss function since they produced the best results
  model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

  return model

In [ ]:
# The model can now be trained using the whole dataset accordingly
model = create_model() # Creating the model

# Training the model with training data
model.fit(X_train, y_train, epochs = 12, batch_size = 4, validation_split = 0.2)

In [ ]:
# To check of it's performance using classfication metrics, the classfication report function is utilized
from sklearn.metrics import classification_report

predictions = np.argmax(model.predict(X_test), axis = 1) # Predictions of the model on the testing data is made
# The report is shown which consists of Precision, Recall, F1-Score and Accuracy
print(classification_report(y_test, predictions))

In [ ]:
# Saving the model in the h5 file format so that it can be deployed
model.save('model.h5')